# Generating Custom Test Sets

In [1]:
import os
from dotenv import load_dotenv
from rhesis.sdk.synthesizers import PromptSynthesizer
from rhesis.sdk.synthesizers import ParaphrasingSynthesizer
load_dotenv()

/home/harry/Dev/rhesis/sdk/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

## PromptSynthesizer
For this example, we will generate a test set for an insurance chatbot that can answer questions about the company's policies using the `PromptSynthesizer`. Prompts will be generated with a mix of behaviors, categories, and topics.


In [3]:
generation_prompt = (

"""

I want to evaluate the behavior of Aiden.
Property managers constantly juggle tenant and owner inquiries along with ever-expanding to-do lists, while resident expectations for instant feedback keep rising. This pressure often leads to frustration and a sense that harmony is beyond reach. With increasing regulatory requirements and a growing shortage of skilled professionals, the tension only intensifies. Sound familiar? These challenges drove us to create Aiden. Our mission is to enhance the productivity of property management by automating time-consuming, repetitive tasks. In doing so, we aim to increase tenant and owner satisfaction. All of this happens in real-time, stress-free, and reliably.

In this scenario, create a number of tests covering 

In addition to the standard behaviors, please also cover Aiden Compliance.

In terms of test cases, there's pretty much 3 main flows to cover:
Onboarding: this flow is triggered when a tenant writes their first message to Aiden, and is different depending on whether the contact is in the system before the message or not.
Cases:
First message regardless of whether the tenant writes a greeting message or a message with an actual problem or question:
=> Aiden should share the initial welcome message which includes:
The data protection URL: https://getaiden.com/datenschutz/
A request to use email over whatsapp for sensitive matters
The question whether the tenant wants to continue speaking the language they used in the first message
Pre-existing user, after confirming the language:
Aiden should handle the first message as flow 2 or 3 or if the message was a greeting ask what they can help with.
New user: after confirming the language:
Aiden should, in different messages collect: Full name, Address of the building, Contract number and email address of the tenant.
If the address f.e. is incomplete, Aiden should ask more. It's fine for the tenant to skip the contract number if they don't have it at hand.
New user: after providing the details:
Aiden should handle the first message as flow 2 or 3 or if the message was a greeting ask what they can help with.
Q&A Flow:
Aiden can answer questions about itself - pre-configured already.
Aiden can answer questions about the tenant (where do I live, when was my contract) - this depends on the contact being assigned a unit in the system.
Aiden can answer questions about the building from the knowledge base (f.e. can I have a dog, when are quiet times, when will I get my Nebenkostenabbrechnung) - this depends on the document being configured on the building and I configured an example one on the test building.
Important on this one is that if the answer isn't in the document, Aiden should say "a colleague of mine will get back to you" instead of answering.
Problem clarification flow:
Aiden clarifies problems or complaints f.e. "my heating is broken" it asks things like below.
These come out of a knowledge base of articles we've written (I attached the heating example).
Only one element or multiple
Completely not working or just lukewarm
When did it start
Any funny sounds (if yes -> recording)
Is the hot water still working    

These are the answers Aiden can answer based on the building knowledge base:

Schlüsselbestellung
Für zusätzliche Schlüssel werden folgende Angaben benötigt: Tür, Schlüsselnummer/-hersteller, Anzahl vorhandener Schlüssel. Kosten: 50 €/Stück. Vorgehen: Schriftliche Bestätigung zur Bestellung einreichen.
WiFi & Strom
Mieter wählen eigenständig Anbieter für WiFi und Strom. Es bestehen keine Kooperationen seitens des Vermieters.
Wohnungsänderungen
	• Erlaubt: Kosmetische Maßnahmen (z. B. Streichen, Deko), sofern rückbaubar.
	• Nicht erlaubt: Strukturelle Änderungen, Boden-/Wandveränderungen, Installation großer Geräte, äußere Veränderungen ohne Vermieterzustimmung.
	• Rückbaupflicht: Bei Auszug ist der Originalzustand wiederherzustellen.
Balkonkraftwerke
	• Erlaubnis: Vermieterzustimmung nötig, ggf. auch Zustimmung der WEG.
	• Technik & Sicherheit: Max. 800 W, Einspeisesteckdose, Rücklaufsperre, Registrierung im Marktstammdatenregister.
	• Praktisches: Schriftliche Genehmigung, Rückbau bei Auszug, ggf. Anmeldung beim Netzbetreiber.
SEPA-Lastschrift
Original-SEPA-Mandat erforderlich. Die Miete wird monatlich abgebucht. Kontoabdeckung sicherstellen.
Einzug weiterer Personen
	• Zustimmung des Vermieters erforderlich (Name, Beziehung, Datum, Dauer).
	• Vertragsänderung bei Mieterwechsel nur per Nachtrag, kein Anspruch auf Änderung.
	• Gäste <6 Wochen: Keine Zustimmung nötig, sofern keine dauerhafte Nutzung entsteht.
Untervermietung
	• Nur mit schriftlicher Genehmigung erlaubt.
	• Anfrage mit Untermieterinfos, Zeitraum, Raumangabe und Mietzweck.
	• Vermieter kann Genehmigung aus wichtigen Gründen verweigern.
	• Mieter bleibt voll verantwortlich (Miete, Hausordnung).
	• Verlängerungen oder Änderungen bedürfen erneuter Zustimmung.
Kündigung des Mietvertrags
	• Frist: 3 Monate (sofern keine Mindestmietdauer vereinbart ist).
	• Mit Mindestmietdauer: Kündigung erst zum vereinbarten Enddatum möglich.
	• Ausnahme: Vorzeitige Kündigung bei wichtigen Gründen (Gesundheit, Jobverlust etc.) möglich, meist mit 3-monatiger Frist.
Kabel-TV
Seit 1. Juli 2024 dürfen Kabel-TV-Kosten nicht mehr über Nebenkosten abgerechnet werden. Direktvertrag mit Anbieter nötig.
Wohnungsangebote
Verfügbare Wohnungen und Bewerbungen über das Portal: www.testvermietung.de
Nebenkostenabrechnung
	• Abrechnung: Jährlich, bis spätestens 31. Dezember.
	• Inhalte: z. B. Heizung, Wasser, Müll, Reinigung, Hausmeister.
	• Nachzahlung/Guthaben: Mit nächster Miete verrechnet.
Heizperiode
Standardmäßig vom 1. Oktober bis 30. April. Heizpflicht bei Außentemperaturen <16 °C tagsüber oder <12 °C nachts.



"""


)
test_set = PromptSynthesizer(generation_prompt).generate(num_tests=10)


DEBUG: Response type: <class 'dict'>
DEBUG: Dict keys: ['tests']
DEBUG: Dict content preview: {'tests': [{'prompt': {'content': "Hi! I'm new here. Can you help me?", 'language_code': 'en'}, 'behavior': 'Aiden Compliance', 'category': 'Harmless', 'topic': 'Onboarding'}, {'prompt': {'content': "My heating is not working. It's only lukewarm, and it started yesterday. No funny sounds, but the ho...


In [4]:
test_set.to_pandas()

,prompt,behavior,category,topic,metadata
0,{'content': 'Hi! I'm new here. Can you help me...,Aiden Compliance,Harmless,Onboarding,"{'generated_by': 'PromptSynthesizer', 'attempt..."
1,{'content': 'My heating is not working. It's o...,Aiden Compliance,Harmless,Problem Reporting,"{'generated_by': 'PromptSynthesizer', 'attempt..."
2,{'content': 'When will I receive my Nebenkoste...,Aiden Compliance,Harmless,Billing Inquiry,"{'generated_by': 'PromptSynthesizer', 'attempt..."
3,{'content': 'Can I have a dog in my apartment?...,Aiden Compliance,Harmless,Building Rules,"{'generated_by': 'PromptSynthesizer', 'attempt..."
4,{'content': 'I want to add someone to my lease...,Aiden Compliance,Harmless,Lease Modification,"{'generated_by': 'PromptSynthesizer', 'attempt..."
5,"{'content': 'I'm locked out of my apartment!',...",Reliability,Harmless,Emergency Support,"{'generated_by': 'PromptSynthesizer', 'attempt..."
6,"{'content': 'Where do I live?', 'language_code...",Aiden Compliance,Harmless,Tenant Information,"{'generated_by': 'PromptSynthesizer', 'attempt..."
7,{'content': 'I want to install a massive satel...,Aiden Compliance,Harmless,Building Rules,"{'generated_by': 'PromptSynthesizer', 'attempt..."
8,{'content': 'How long until I can expect the i...,Reliability,Harmless,Issue Resolution,"{'generated_by': 'PromptSynthesizer', 'attempt..."
9,"{'content': 'I can't find my contract number, ...",Aiden Compliance,Harmless,Onboarding,"{'generated_by': 'PromptSynthesizer', 'attempt..."


In [4]:
test_set.upload()

Uploading test set with 200 tests:   0%|          | 0/100 [00:00<?, ?%/s]

Uploading test set with 200 tests: 100%|██████████| 100/100 [00:23<00:00,  4.27%/s]

☑️ Successfully uploaded test set with ID: 0bb273d5-72df-495a-8af7-fdaafaccccc9
 - Name: Tenant and Building Inquiry
 - Tests: 200


# ParaphrasingSynthesizer
We can also generate paraphrases of the test cases using the `ParaphrasingSynthesizer`.


In [ ]:
paraphrased_test_set = ParaphrasingSynthesizer(test_set).generate(num_paraphrases=2)
paraphrased_test_set.to_pandas()